In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:
from nest_asyncio import apply
apply()

In [4]:
from chat import *
import os

In [5]:
from psql_helpers import (
    async_session, get_session,

)
from vexa import *

In [6]:
auth = VexaAuth()
vexa = VexaAPI()
user_id = (await vexa.get_user_info())['id']

# Initialize search engines

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [7]:
qdrant_engine = QdrantSearchEngine(os.getenv('VOYAGE_API_KEY'))
es_engine = await ElasticsearchBM25.create() 

Connecting to Elasticsearch at elasticsearch:9200


In [8]:
chat_manager = UnifiedChatManager(
        session=get_session(),
        qdrant_engine=qdrant_engine,
        es_engine=es_engine
    )

In [9]:
query = "What is the weather in San Francisco?"

In [10]:
fronten_id = None
thread_id = None
content_id = None
entity_id = None
content_ids = None
entity_ids = None
model = "gpt-4o-mini"
temperature = 0.0
meta = None


In [11]:
from helpers import jupyter_stream_output


In [12]:
from psql_helpers import get_session

@jupyter_stream_output(markdown=True)
async def unified_chat(
    query: str,
    content_id: Optional[UUID] = None,
    entity_id: Optional[int] = None,
    content_ids: Optional[List[UUID]] = None,
    entity_ids: Optional[List[int]] = None,
    thread_id: Optional[str] = None,
    temperature: float = 0.7
):
    """
    Unified chat function supporting both content and entity-based search
    
    Args:
        query: The user's query
        content_id: Optional single content ID for direct thread mapping
        entity_id: Optional single entity ID for direct thread mapping
        content_ids: Optional list of content IDs for search scope
        entity_ids: Optional list of entity IDs for search scope
        thread_id: Optional thread ID to continue conversation
        temperature: Model temperature parameter
    """
    
    async with get_session() as session:
        chat_manager = UnifiedChatManager(
            session=session,
            qdrant_engine=qdrant_engine,
            es_engine=es_engine
        )
        
        async for result in chat_manager.chat(
            user_id=user_id,
            query=query,
            content_id=content_id,
            entity_id=entity_id,
            content_ids=content_ids,
            entity_ids=entity_ids,
            thread_id=thread_id,
            temperature=temperature
        ):
            if 'chunk' in result:
                yield result['chunk']
            elif 'linked_output' in result:
                yield result
                
            print(result)

In [13]:
await unified_chat(query="and the last person you mentioned name is?",content_ids=[UUID('580bb613-00de-4c4a-b048-2c2576fcdef0'),
                                                                         UUID('fb996504-0475-4ee9-a85e-dda4e9f15a4b'),
                                                                         UUID('a0c3f703-2a5e-4ed0-9561-807121516984'),
                                                                              UUID('6a7287fb-b6ae-46cf-8b99-1ce1bcc81403')],thread_id='475e6948-a903-4f63-b067-3084bef1c5fb')



The last person I mentioned is **Artem Puzik**. He participated in the discussion on **January 23, 2025**, where you talked about the integration of an API within an existing application, specifically focusing on the authentication process and handling user tokens. If you need more details about your conversation with him or any other topic, just let me know!

{'chunk': '!'}


{'thread_id': '475e6948-a903-4f63-b067-3084bef1c5fb',
 'output': 'The last person I mentioned is **Artem Puzik**. He participated in the discussion on **January 23, 2025**, where you talked about the integration of an API within an existing application, specifically focusing on the authentication process and handling user tokens. If you need more details about your conversation with him or any other topic, just let me know!',
 'linked_output': 'The last person I mentioned is **Artem Puzik**. He participated in the discussion on **January 23, 2025**, where you talked about the integration of an API within an existing application, specifically focusing on the authentication process and handling user tokens. If you need more details about your conversation with him or any other topic, just let me know!',
 'service_content': {'output': 'The last person I mentioned is **Artem Puzik**. He participated in the discussion on **January 23, 2025**, where you talked about the integration of an API

In [14]:
await unified_chat(query="who is this guy?",entity_id=5432)



The person you're referring to is **Mariana Montenegro**, who introduced herself during the meeting on **January 21, 2025**. She is the manager responsible for the **AI hub** at **Unicorn Factory Lisboa**. Mariana discussed the purpose of the AI hub, emphasizing its community focus and potential for collaboration in the AI startup space, which aligns with your interests in AI and startup ecosystems. If you need more specific information about her role or the AI hub, feel free to ask!

{'chunk': '!'}


{'thread_id': '0396fa00-7514-4d3b-abf2-716fb2fed08c',
 'output': "The person you're referring to is **Mariana Montenegro**, who introduced herself during the meeting on **January 21, 2025**. She is the manager responsible for the **AI hub** at **Unicorn Factory Lisboa**. Mariana discussed the purpose of the AI hub, emphasizing its community focus and potential for collaboration in the AI startup space, which aligns with your interests in AI and startup ecosystems. If you need more specific information about her role or the AI hub, feel free to ask!",
 'linked_output': "The person you're referring to is **Mariana Montenegro**, who introduced herself during the meeting on **January 21, 2025**. She is the manager responsible for the **AI hub** at **Unicorn Factory Lisboa**. Mariana discussed the purpose of the AI hub, emphasizing its community focus and potential for collaboration in the AI startup space, which aligns with your interests in AI and startup ecosystems. If you need more spec